In [1]:
import torchvision as thv
import torch as th
import time
from torchinfo import summary
th.manual_seed(32)
th.cuda.manual_seed(32)
th.cuda.manual_seed_all(32)

In [2]:
DEVICE = "cuda" if th.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [3]:
TRAIN_DATASET_LOC = r"E:\PyTorch\Working Data\Input Data\Elephants\dataset\train"
TEST_DATASET_LOC = r"E:\PyTorch\Working Data\Input Data\Elephants\dataset\test"

In [4]:
train_dataset = thv.datasets.ImageFolder(TRAIN_DATASET_LOC,
                                         transform=thv.transforms.Compose([thv.transforms.Resize(size=(224,224)),thv.transforms.ToTensor()]),
                                         target_transform=None)

test_dataset = thv.datasets.ImageFolder(TEST_DATASET_LOC,
                                        transform=thv.transforms.Compose([thv.transforms.Resize(size=(224,224)),thv.transforms.ToTensor()]),
                                        target_transform=None)

In [5]:
train_data_loader = th.utils.data.DataLoader(train_dataset,
                                             shuffle=True,
                                             batch_size=32)

test_data_loader = th.utils.data.DataLoader(test_dataset,
                                            shuffle=True,
                                            batch_size=32)

In [4]:
from torch import nn

model = nn.Sequential(
                            nn.Conv2d(in_channels=3,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                            nn.ReLU(),
                            nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2),
                            nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                            nn.ReLU(),
                            nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2),
                            nn.Flatten(),
                            nn.Linear(in_features=28090,out_features=1) 
)

In [7]:
model

Sequential(
  (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (1): ReLU()
  (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (6): ReLU()
  (7): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=28090, out_features=1, bias=True)
)

In [8]:
from torchinfo import summary

summary(model,
        input_size=(1,3,224,224),
        verbose=1,
        col_names=["input_size","output_size","num_params","trainable"],
        row_settings=["var_names"],
        col_width=18);

Layer (type (var_name))                  Input Shape        Output Shape       Param #            Trainable
Sequential (Sequential)                  [1, 3, 224, 224]   [1, 1]             --                 True
├─Conv2d (0)                             [1, 3, 224, 224]   [1, 10, 222, 222]  280                True
├─ReLU (1)                               [1, 10, 222, 222]  [1, 10, 222, 222]  --                 --
├─Conv2d (2)                             [1, 10, 222, 222]  [1, 10, 220, 220]  910                True
├─ReLU (3)                               [1, 10, 220, 220]  [1, 10, 220, 220]  --                 --
├─MaxPool2d (4)                          [1, 10, 220, 220]  [1, 10, 110, 110]  --                 --
├─Conv2d (5)                             [1, 10, 110, 110]  [1, 10, 108, 108]  910                True
├─ReLU (6)                               [1, 10, 108, 108]  [1, 10, 108, 108]  --                 --
├─Conv2d (7)                             [1, 10, 108, 108]  [1, 10, 106, 106

In [5]:
loss_fn = th.nn.BCEWithLogitsLoss()

optimizer = th.optim.Adam(model.parameters(),
                          lr=0.001)

In [6]:
def accuracy(y_preds,y_true):
    correct = th.eq(y_preds,y_true).sum().item()
    acc = (correct/len(y_preds)) * 100
    return acc

In [ ]:
EPOCHS = 30
TRAIN_DL_LEN = len(train_data_loader)
TEST_DL_LEN = len(test_data_loader)

model.to(DEVICE)
for epoch in range(EPOCHS):
    start = time.time() 
    model.train()
    epoch_train_loss = 0.0
    epoch_train_accuracy = 0.0
    for X,Y in train_data_loader:
        X = X.to(DEVICE)
        Y = Y.type(th.float32).to(DEVICE)
        
        y_logits = model(X).squeeze()
        y_preds = th.round(th.sigmoid(y_logits))
        
        loss = loss_fn(y_logits,Y)
        acc = accuracy(y_preds,Y)
        
        epoch_train_loss += loss
        epoch_train_accuracy += acc
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    model.eval()
    epoch_test_loss = 0.0
    epoch_test_accuracy = 0.0
    with th.no_grad():
        for X,Y in test_data_loader:
            X = X.to(DEVICE)
            Y = Y.type(th.float32).to(DEVICE)
            
            y_logits = model(X).squeeze()
            y_preds = th.round(th.sigmoid(y_logits))
            
            loss = loss_fn(y_logits,Y)
            acc = accuracy(y_preds,Y)
            
            epoch_test_loss += loss
            epoch_test_accuracy += acc
    
    epoch_train_loss /= TRAIN_DL_LEN
    epoch_train_accuracy /= TRAIN_DL_LEN
    epoch_test_loss /= TEST_DL_LEN
    epoch_test_accuracy /= TEST_DL_LEN
    end = time.time()
    print(f"Epoch {epoch}[{end-start}] :- |Train Loss:- {epoch_train_loss:.2f} <--------> Train Accuracy:- {epoch_train_accuracy:.2f}|++++++++|Test Loss:- {epoch_test_loss:.2f} <--------> Test Accuracy :- {epoch_test_accuracy:.2f}|")

Epoch 0[6.033707618713379] :- |Train Loss:- 0.70 <--------> Train Accuracy:- 51.27|++++++++|Test Loss:- 0.70 <--------> Test Accuracy :- 48.29|
Epoch 1[5.779543399810791] :- |Train Loss:- 0.69 <--------> Train Accuracy:- 54.17|++++++++|Test Loss:- 0.66 <--------> Test Accuracy :- 62.80|
Epoch 2[5.9641807079315186] :- |Train Loss:- 0.68 <--------> Train Accuracy:- 58.80|++++++++|Test Loss:- 0.66 <--------> Test Accuracy :- 59.00|
Epoch 3[5.849159240722656] :- |Train Loss:- 0.68 <--------> Train Accuracy:- 57.75|++++++++|Test Loss:- 0.67 <--------> Test Accuracy :- 57.74|
Epoch 4[5.746140241622925] :- |Train Loss:- 0.66 <--------> Train Accuracy:- 59.49|++++++++|Test Loss:- 0.63 <--------> Test Accuracy :- 61.61|
Epoch 5[5.819441795349121] :- |Train Loss:- 0.66 <--------> Train Accuracy:- 61.11|++++++++|Test Loss:- 0.64 <--------> Test Accuracy :- 61.98|
Epoch 6[5.7381627559661865] :- |Train Loss:- 0.65 <--------> Train Accuracy:- 62.96|++++++++|Test Loss:- 0.63 <--------> Test Accuracy 

In [7]:
from torchvision.transforms import v2

train_dataset1 = thv.datasets.ImageFolder(root=TRAIN_DATASET_LOC,
                                        transform=v2.Compose([v2.ToImage(),
                                                              v2.Resize(size=[224,224],antialias=True),
                                                              v2.ToDtype(th.float32,scale=True)]))

test_dataset1 = thv.datasets.ImageFolder(root=TEST_DATASET_LOC,
                                        transform=v2.Compose([v2.ToImage(),
                                                              v2.Resize(size=[224,224],antialias=True),
                                                              v2.ToDtype(th.float32,scale=True)]))

In [8]:
train_data_loader1 = th.utils.data.DataLoader(train_dataset1,
                                             shuffle=True,
                                             batch_size=32)

test_data_loader1 = th.utils.data.DataLoader(test_dataset1,
                                            shuffle=True,
                                            batch_size=32)

In [9]:
EPOCHS = 30
TRAIN_DL_LEN = len(train_data_loader1)
TEST_DL_LEN = len(test_data_loader1)

model.to(DEVICE)
for epoch in range(EPOCHS):
    start = time.time() 
    model.train()
    epoch_train_loss = 0.0
    epoch_train_accuracy = 0.0
    for X,Y in train_data_loader1:
        X = X.to(DEVICE)
        Y = Y.type(th.float32).to(DEVICE)
        
        y_logits = model(X).squeeze()
        y_preds = th.round(th.sigmoid(y_logits))
        
        loss = loss_fn(y_logits,Y)
        acc = accuracy(y_preds,Y)
        
        epoch_train_loss += loss
        epoch_train_accuracy += acc
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    model.eval()
    epoch_test_loss = 0.0
    epoch_test_accuracy = 0.0
    with th.no_grad():
        for X,Y in test_data_loader1:
            X = X.to(DEVICE)
            Y = Y.type(th.float32).to(DEVICE)
            
            y_logits = model(X).squeeze()
            y_preds = th.round(th.sigmoid(y_logits))
            
            loss = loss_fn(y_logits,Y)
            acc = accuracy(y_preds,Y)
            
            epoch_test_loss += loss
            epoch_test_accuracy += acc
    
    epoch_train_loss /= TRAIN_DL_LEN
    epoch_train_accuracy /= TRAIN_DL_LEN
    epoch_test_loss /= TEST_DL_LEN
    epoch_test_accuracy /= TEST_DL_LEN
    end = time.time()
    print(f"Epoch {epoch}[{end-start}] :- |Train Loss:- {epoch_train_loss:.2f} <--------> Train Accuracy:- {epoch_train_accuracy:.2f}|++++++++|Test Loss:- {epoch_test_loss:.2f} <--------> Test Accuracy :- {epoch_test_accuracy:.2f}|")

Epoch 0[5.818410634994507] :- |Train Loss:- 0.70 <--------> Train Accuracy:- 50.23|++++++++|Test Loss:- 0.69 <--------> Test Accuracy :- 51.64|
Epoch 1[5.507781267166138] :- |Train Loss:- 0.69 <--------> Train Accuracy:- 55.21|++++++++|Test Loss:- 0.67 <--------> Test Accuracy :- 59.08|
Epoch 2[5.728950500488281] :- |Train Loss:- 0.68 <--------> Train Accuracy:- 59.72|++++++++|Test Loss:- 0.65 <--------> Test Accuracy :- 61.68|
Epoch 3[5.999970197677612] :- |Train Loss:- 0.68 <--------> Train Accuracy:- 56.94|++++++++|Test Loss:- 0.65 <--------> Test Accuracy :- 65.03|
Epoch 4[5.8573899269104] :- |Train Loss:- 0.67 <--------> Train Accuracy:- 59.03|++++++++|Test Loss:- 0.64 <--------> Test Accuracy :- 64.14|
Epoch 5[5.799300670623779] :- |Train Loss:- 0.66 <--------> Train Accuracy:- 60.53|++++++++|Test Loss:- 0.63 <--------> Test Accuracy :- 63.84|
Epoch 6[5.577592134475708] :- |Train Loss:- 0.65 <--------> Train Accuracy:- 62.15|++++++++|Test Loss:- 0.63 <--------> Test Accuracy :- 6